# Create mixed patch files

Eventually we'll process full images directly while creating patches on the fly.  
Here, save all patches to files.
From 21 GB of raw images, writes 169 MB of patches.

In [1]:
import time
import os
#  os.environ['CUDA_VISIBLE_DEVICES'] = "0"   # turns off GPU?
import glob
import cv2 # OpenCV-Python
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import json
from tensorflow import keras
import keras.layers as kl
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
DIR_IMAGES_RAW = "/home/jrm/Martinez/images/raw/"
DIR_IMAGES_OUT = "/home/jrm/Martinez/images/patches/"
PATTERN_IMAGES_RAW = "*.DF1.*.tif"
PATCH_SIZE=224  # matches VGG
DF_Ypos = ['B7.','B15','D1.','D5.','E7.','E9.','F9.','G3.','H13','I1.','I5.','I13']
DF_Yneg = ['A3.','A5.','B13','C1.','C11','D3.','E5.','F3.','F7','F11','F13','F15','G15','H1.','H3.','H7.','H15']

In [3]:
def get_image_names(path,pattern):
    #RAW_IMAGE_NAMES = os.listdir(DIR_IMAGES_RAW)
    paths = glob.glob(path+pattern)
    names = [os.path.basename(x) for x in paths]
    return names
FILENAMES_IMAGES_RAW = get_image_names(DIR_IMAGES_RAW,PATTERN_IMAGES_RAW)

In [6]:
class patch_maker:
    def __init__(self):
        self.path=""
        self.w=0
        self.h=0
        self.im_width = 0
        self.im_height = 0
        self.patch_size = 10 # scalar, assumed square for now
        self.pixel_array = None
        self.output_format='.jpg'
    def set_input_path(self,path):
        self.path=path
    def set_patch_size(self,scalar):
        self.patch_size = scalar
    def load_pixel_array(self,filename,verbose=False):
        self.w=0
        self.h=0
        im = Image.open(self.path+filename)
        ima = np.array(im)   # convert to numpy
        self.im_width = ima.shape[0]
        self.im_height = ima.shape[1]    
        if verbose:
            print(filename, ima.size, ima.shape)
        self.pixel_array = ima
        return ima
    def get_next_patch(self): 
        IM_WIDTH = self.im_width
        IM_HEIGHT = self.im_height
        PIXEL_ARRAY = self.pixel_array
        patch = None
        if self.w+self.patch_size > IM_WIDTH:
            self.h += self.patch_size
            self.w = 0
        if self.w+self.patch_size <= IM_WIDTH and self.h+self.patch_size <= IM_HEIGHT:
            patch = PIXEL_ARRAY[self.w:self.w+self.patch_size, self.h:self.h+self.patch_size]
            self.w += self.patch_size
        return patch
    def save_patch (self, prefix, pnum, patch):
        ext = self.output_format
        path=DIR_IMAGES_OUT+prefix+'.'+str(pnum)+ext
        im = Image.fromarray(patch)
        im.save(path)

In [9]:
NO_GREEN_INTENSITY=np.array( [0,35,0] )
NO_BLUE_INTENSITY=np.array( [0,0,30] )
BACKGROUND_MAX=np.array( [20,20,20] )
PATCH_THRESHOLD=PATCH_SIZE*PATCH_SIZE/2
pm = patch_maker()
pm.set_input_path(DIR_IMAGES_RAW)
pm.set_patch_size(PATCH_SIZE)
for filename in FILENAMES_IMAGES_RAW:
    prefix = filename[0:3]
    pnum = 0
    blue = 0
    black = 0
    green = 0
    if prefix in DF_Yneg:
        print(filename,end=" ")
        pixels = pm.load_pixel_array(filename)
        patch = pm.get_next_patch()
        while patch is not None:
            num_green_pixels = np.sum(np.all(patch >= NO_GREEN_INTENSITY,axis=2))
            num_blue_pixels = np.sum(np.all(patch >= NO_BLUE_INTENSITY,axis=2))
            num_background_pixels = np.sum(np.all(patch<BACKGROUND_MAX,axis=2))
            if num_background_pixels>=PATCH_THRESHOLD:
                black += 1
            if num_blue_pixels>=10000 and num_blue_pixels>num_green_pixels*20:
                blue += 1
            elif num_green_pixels>=15000 and num_green_pixels>num_blue_pixels*2:
                green += 1
            else:
                pnum += 1
                pm.save_patch(prefix,pnum,patch)
            patch = pm.get_next_patch()
        print(" Filter%d black, %d blue, %d green. Write %d pixels."%(black,blue,green,pnum))
print('done')

F15.DF1.135.tif  Filter1873 black, 262 blue, 34 green. Write 2839 pixels.
H3.DF1.27.tif  Filter1509 black, 30 blue, 110 green. Write 2995 pixels.
C11.DF1.96.tif  Filter1507 black, 16 blue, 292 green. Write 2827 pixels.
C1.DF1.01.tif  Filter1735 black, 293 blue, 69 green. Write 2773 pixels.
F11.DF1.94.tif  Filter1740 black, 375 blue, 54 green. Write 2706 pixels.
H7.DF1.62.tif  Filter1849 black, 228 blue, 178 green. Write 2729 pixels.
A3.DF1.26.tif  Filter1483 black, 1482 blue, 1 green. Write 1652 pixels.
A5.DF1.45.tif  Filter889 black, 6 blue, 454 green. Write 2675 pixels.
H15.DF1.132.tif  Filter1169 black, 0 blue, 66 green. Write 3069 pixels.
G15.DF1.133.tif  Filter1902 black, 0 blue, 239 green. Write 2896 pixels.
F3.DF1.22.tif  Filter1841 black, 41 blue, 430 green. Write 2664 pixels.
B13.DF1.115.tif  Filter1608 black, 407 blue, 70 green. Write 2658 pixels.
D3.DF1.20.tif  Filter1685 black, 4 blue, 325 green. Write 2806 pixels.
E5.DF1.42.tif  Filter1474 black, 266 blue, 31 green. Write 